<a href="https://colab.research.google.com/github/TAUforPython/BioMedAI/blob/main/LLM_med_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Генерация диалога с помощью RuDlg-Mistral

https://github.com/pineforest-ai/nlp_examples/tree/main

In [1]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [45]:
#BASE_MODEL = "Open-Orca/Mistral-7B-SlimOrca"
BASE_MODEL = "Mykes/rubert_ner_SDDCS"
#BASE_MODEL = "cointegrated/rubert-tiny2"

В контексте модели Mistral или других языковых моделей (LLM), adapter_model — это дополнительный модуль или слой, который можно "подключить" к основной модели для адаптации её под конкретные задачи или домены без необходимости полного переобучения модели.

Что такое адаптеры?

Адаптеры — это небольшие, легковесные модули, которые добавляются к слоям основной модели. Они обучаются на специфических данных, позволяя модели лучше справляться с конкретными задачами (например, классификация текста, перевод, анализ тональности и т.д.), при этом основная модель остаётся неизменной. Это особенно полезно, когда у вас ограничены вычислительные ресурсы или данные для полного переобучения модели.

offload_folder - нужен предотвращения ошибки переполнения дискового места окружения.

In [12]:
import os
from huggingface_hub import login
from google.colab import userdata

os.environ["HF_token"] = userdata.get("HF_token")

login(os.environ["HF_token"])

Тонкая настройка больших предварительно обученных моделей часто оказывается слишком затратной из-за их масштаба. Методы эффективной тонкой настройки параметров (Parameter-Efficient Fine-Tuning, PEFT) позволяют эффективно адаптировать большие предварительно обученные модели к различным прикладным задачам путем тонкой настройки лишь небольшого количества дополнительных параметров модели вместо всех её параметров. Это значительно снижает вычислительные и стоимостные затраты. Современные передовые методы PEFT демонстрируют результаты, сопоставимые с полностью дообученными моделями.

PEFT интегрирован с библиотеками Transformers для упрощения обучения и вывода моделей, Diffusers для удобного управления различными адаптерами и Accelerate для распределенного обучения и вывода при работе с действительно большими моделями.

In [22]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Mykes/rubert_ner_SDDCS")
model = AutoModelForTokenClassification.from_pretrained("Mykes/rubert_ner_SDDCS")
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12,

In [32]:
from transformers import pipeline

pipe = pipeline(task="ner", model='Mykes/rubert_ner_SDDCS', tokenizer='Mykes/rubert_ner_SDDCS', aggregation_strategy="max")
# I made the misspelled words on purpose
query = "У ребенка треога и норушения сна, потеря сознания, раньше ставили паническое расстройство. Подскажи психиатра в районе метро Октбрьской."
pipe(query.lower())

Device set to use cpu


[{'entity_group': 'AGE',
  'score': 0.9990075,
  'word': 'ребенка',
  'start': 2,
  'end': 9},
 {'entity_group': 'SYMPTOM',
  'score': 0.9982822,
  'word': 'треога',
  'start': 10,
  'end': 16},
 {'entity_group': 'SYMPTOM',
  'score': 0.9274422,
  'word': 'норушения сна',
  'start': 19,
  'end': 32},
 {'entity_group': 'SYMPTOM',
  'score': 0.99990046,
  'word': 'потеря сознания',
  'start': 34,
  'end': 49},
 {'entity_group': 'DISEASE',
  'score': 0.99987423,
  'word': 'паническое расстройство',
  'start': 66,
  'end': 89},
 {'entity_group': 'SUBWAY',
  'score': 0.96067816,
  'word': 'октбрьской',
  'start': 125,
  'end': 135}]

In [26]:
import spacy
from spacy import displacy

def convert_to_displacy_format(text, ner_results):
    entities = []
    for result in ner_results:
        # Convert the Hugging Face output into the format displacy expects
        entities.append({
            "start": result['start'],
            "end": result['end'],
            "label": result['entity_group']
        })
    return {
        "text": text,
        "ents": entities,
        "title": None
    }
query = "У ребенка треога и норушения сна, потеря сознания, раньше ставили паническое расстройство, принимал атаракс. Подскажи хорошего психиатра в районе метро Октбрьской."
ner_results = pipe(query.lower())
displacy_data = convert_to_displacy_format(query, ner_results)
colors = {
    "SPECIALITY": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "CITY": "linear-gradient(90deg, #feca57, #ff9f43)",
    "DRUG": "linear-gradient(90deg, #55efc4, #81ecec)",
    "DISEASE": "linear-gradient(90deg, #fab1a0, #ff7675)",
    "SUBWAY": "linear-gradient(90deg, #00add0, #0039a6)",
    "AGE": "linear-gradient(90deg, #f39c12, #e67e22)",
    "SYMPTOM": "linear-gradient(90deg, #e74c3c, #c0392b)"
}
options = {"ents": ["SPECIALITY", "CITY", "DRUG", "DISEASE", "SYMPTOM", "AGE", "SUBWAY"], "colors": colors}
html = displacy.render(displacy_data, style="ent", manual=True, options=options, jupyter=False)
with open("ner_visualization_with_colors.html", "w", encoding="utf-8") as f:
    f.write(html)
from IPython.display import display, HTML
display(HTML(html))

Named Entity Recognition (NER) model based on the cointegrated/rubert-tiny2 model with only 29.4M params, designed to detect russian medical entities like diseases, drugs, symptoms